In [ ]:
from trips.testProblems import *
from pylops import Identity, LinearOperator
from trips.utils import operator_qr, operator_svd, is_identity
from trips.parameter_selection.gcv import *
import scipy.linalg as la
import scipy.optimize as op
import numpy as np
#from trips.solvers.Tikhonov import *
from trips.solvers.Hybrid_LSQR import *

In [ ]:
D1D = Deblurring1D()
N = 64
n = N
A = D1D.forward_Op_matrix_1D(3, N)
# A = np.vstack((A, np.random.rand(3,N)))
x_true = D1D.gen_xtrue(N, test = 'piecewise')
btrue = A@x_true
(b, delta) = D1D.add_noise(btrue, 'Gaussian', 0.05)
plt.plot(x_true)
D1D.plot_data(b)

In [ ]:
print(delta)
deltavar = la.norm(b.reshape((-1,1)) - (A@x_true).reshape((-1,1)))
print(deltavar)

In [ ]:
import scipy.optimize as op
import scipy.linalg as la
from trips.utils import operator_qr, operator_svd, is_identity
import warnings
def discrepancy_principle(A, b, L, delta = None, eta = 1.01, **kwargs):

    if not ( isinstance(delta, float) or isinstance(delta, int)):

        # raise TypeError('You must provide a value for the noise level delta.')
        raise Warning("""A value for the noise level delta was not provided. A default value of 0.01 has been used. 
                    Please supply a value of delta based on the estimated noise level of the problem.""")

        delta = 0.01
    print(delta)
    if is_identity(L):
        U, S, V = la.svd(A, full_matrices=False)
        singular_values = S**2
        singular_values.shape = (singular_values.shape[0], 1)
        bhat = U.T @ b
        beta = 1.0

        alpha = 0.0

        iterations = 0

        while (iterations < 30) or ((iterations <= 100) and (np.abs(alpha) < 10**(-16))):

            f = ((singular_values*beta + 1)**(-2)).T @ bhat - (eta*delta)**2

            f_prime = -2*  ((singular_values*beta + 1)**(-3) * singular_values).T @ bhat

            beta_new = beta - f/f_prime


            if abs(beta_new - beta) < 10**(-7)* beta:
                break

            beta = beta_new

            alpha = 1/beta_new

            iterations += 1
    else:
        # n = A.shape[1]
        tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((A, np.sqrt(reg_param)*L)), np.vstack((b.reshape((-1,1)), np.zeros((L.shape[0],1)))))[0]
        discr_func_zero = lambda reg_param: np.linalg.norm(np.matmul(A,tikh_sol(reg_param)).reshape((-1,1)) - b.reshape((-1,1))) - (eta*delta)
        # discr_func_zero = lambda reg_param: np.absolute(np.linalg.norm(np.matmul(A,tikh_sol(reg_param)) - b.reshape((-1,1))) - (eta*delta))
        # alpha = op.fminbound(func = discr_func_zero, x1 = 0, x2 = 1e2, args=(), xtol=1e-12, maxfun=1000, full_output=0, disp=1)
        alpha = op.fsolve(discr_func_zero, 1e-8)[0]

    return {'x':alpha}

In [ ]:
tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((A, np.sqrt(reg_param)*L)), np.vstack((b.reshape((-1,1)), np.zeros((L.shape[0],1)))))[0]
discr_func_zero = lambda reg_param: np.linalg.norm(np.matmul(A,tikh_sol(reg_param)) - b.reshape((-1,1))) - (1.01*deltavar)
xx  = np.logspace(-5,2,200)
yy  = [discr_func_zero(val) for val in xx]
# plt.semilogx(xx, yy)
plt.plot(xx, yy)

In [ ]:
print(yy[0])

In [ ]:
print(deltavar)

In [ ]:
from trips.parameter_selection.gcv import generalized_crossvalidation
# from trips.parameter_selection.discrepancy_principle import discrepancy_principle
def Tikhonov(A, b, L, x_true, regparam = 'gcv', **kwargs):
    if regparam in ['gcv', 'GCV', 'Gcv']:
        lambdah = generalized_crossvalidation(A, b, L) # find ideal lambda by crossvalidation ###
        xTikh = np.linalg.solve(A.T@A + lambdah*L.T@L, A.T@b)
    elif regparam in ['DP', 'dp', 'Dp', 'Discrepancy Principle', 'Discrepancy principle', 'discrepancy principle']:
        lambdah = discrepancy_principle(A, b, L, **kwargs)['x'].item() # find ideal lambdas by discrepancy principle
        print(lambdah)
        xTikh = np.linalg.solve(A.T@A + (lambdah**2)*L.T@L, A.T@b.reshape((-1,1)))
    else:
        lambdah = regparam
        xTikh = np.linalg.solve(A.T@A + lambdah*L.T@L, A.T@b)
    return xTikh, lambdah

In [ ]:
b_true = A@x_true
#delta_new = la.norm(b.reshape((-1,1)) - b_true.reshape((-1,1)))
delta_new = la.norm(b.reshape((-1,1)) - btrue.reshape((-1,1)))
print(delta_new)

In [ ]:
print(delta_new)

In [ ]:
L = np.eye(N)
outTikhI = Tikhonov(A, b, L, x_true, regparam = 'dp', delta = delta_new)

In [ ]:
xTikhI = outTikhI[0]
print(la.norm(A@xTikhI - b.reshape((-1,1)))-1.01*delta_new)

In [ ]:
diags = [-np.ones((N,)), np.ones((N-1,))] 
L = sparse.diags(diags, [0, 1]).toarray()
outTikhL = Tikhonov(A, b, L, x_true, regparam = 'dp', delta = delta_new)

In [ ]:
xTikhL = outTikhL[0]
print(la.norm(A@xTikhL - b.reshape((-1,1))) - 1.01*delta_new)

In [ ]:
plt.plot(x_true)
# plt.plot(outTikhL[0])
plt.plot(outTikhI[0])

In [ ]:
tikh_sol = lambda reg_param: np.linalg.lstsq(np.vstack((A, np.sqrt(reg_param)*L)), np.vstack((b.reshape((-1,1)), np.zeros((n,1)))))[0]
discr_func_zero = lambda reg_param: np.absolute(np.linalg.norm(np.matmul(A,np.linalg.lstsq(np.vstack((A, np.sqrt(reg_param)*L)), np.vstack((b.reshape((-1,1)), np.zeros((n,1)))))[0]) - b) - (1.01*delta_new))
discr_func_var = lambda reg_param: np.linalg.norm(np.matmul(A,tikh_sol(reg_param)) - b.reshape((-1,1))) - (1.01*delta_new)
# alpha = op.fminbound(func = discr_func_zero, x1 = 0, x2 = 1e2, args=(), xtol=1e-12, maxfun=1000, full_output=0, disp=1)
# print(discr_func_var(0))
# print(discr_func_var(1e2))
# alphavar = op.root_scalar(discr_func_var, bracket = [0, 1e2])

In [ ]:
import scipy.io
mat = scipy.io.loadmat('deblur1D.mat')
A = mat['A']
x_true = D1D.gen_xtrue(N, test = 'piecewise')
btrue = A@x_true
(b, delta) = D1D.add_noise(btrue, 'Gaussian', 0.05)
print(A.shape)

In [ ]:
L = np.eye(n)
breg = np.vstack((b.reshape((-1,1)), np.zeros((n,1))))

In [ ]:
lambda1 = 1e-8
Areg1 = np.vstack((A, np.sqrt(lambda1)*L))
x1 = np.linalg.lstsq(Areg1, breg)[0]
print(la.norm(A@x1 - b.reshape((-1,1))) - 1.01*delta_new)
print(discr_func_var(lambda1))

In [ ]:
lambda2 = 1e-2
Areg2 = np.vstack((A, np.sqrt(lambda2)*L))
x2 = np.linalg.lstsq(Areg2, breg)[0]
# print(x2.shape)
# print((A@x2).shape)
# print(b.reshape((-1,1)).shape)
# print((A@x2 - b.reshape((-1,1))).shape)
print(la.norm(A@x2 - b.reshape((-1,1)))- 1.01*delta_new)
print(discr_func_var(lambda2))

In [ ]:
lambda3 = 1e2
Areg3 = np.vstack((A, np.sqrt(lambda3)*L))
x3 = np.linalg.lstsq(Areg3, breg)[0]
print(la.norm(A@x3 - b.reshape((-1,1))) -  1.01*delta_new)
print(discr_func_var(lambda3))

In [ ]:
x1 = np.linalg.lstsq(Areg, breg)[0]

In [ ]:
x2 = np.linalg.lstsq(Areg, breg)[0]

In [ ]:
x3 = np.linalg.lstsq(Areg, breg)[0]

In [ ]:
plt.plot(x1)
plt.plot(x2)
plt.plot(x3)

In [ ]:
d1 = discr_func_var(1e-8)
d2 = discr_func_var(1e-2)
d3 = discr_func_var(1e2)
print(d1)
print(d2)
print(d3)
d1var = la.norm(A@x1 - b) - (1.01*delta_new)
print(d1var)
d2var = la.norm(A@x2 - b) - (1.01*delta_new)
print(d2var)
d3var = la.norm(A@x3 - b) - (1.01*delta_new)
print(d3var)

In [ ]:
xx  = np.logspace(-5,2,200)
yy  = [discr_func_var(val) for val in xx]
# plt.semilogx(xx, yy)
plt.plot(xx, yy)

In [ ]:
print(alpha)

In [2]:
import numpy as np
R_A = np.random.rand(4,4)
np.vstack( [R_A]+ [np.eye(4)] )

array([[0.30598006, 0.02330494, 0.26712049, 0.75797201],
       [0.12203232, 0.98633702, 0.64415331, 0.81496524],
       [0.50697451, 0.95224474, 0.25975989, 0.21776423],
       [0.36372135, 0.20687843, 0.56433954, 0.33748652],
       [1.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])